# Deciding whether to escalate a customer support issue

## Part 1: Load and examine the data

In [3]:
!pip install -r requirements.txt


     |████████████████████████████████| 78 kB 7.9 MB/s  eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [4]:
data_bucket = '' #we should use SageMaker default s3 bucket
subfolder = 'ch04' #we need cha04 inbound.csv datasets
dataset = 'inbound.csv'

In [ ]:
import pandas as pd                               
import boto3
import sagemaker
from sklearn.model_selection import train_test_split
import json
from slugify import slugify
import csv
from time import sleep

role = sagemaker.get_execution_role()
sess = sagemaker.session.Session()
data_bucket = sess.default_bucket()

print(f'SageMaker exectuion role: {role},  default s3 bucket: {data_bucket}')

In [8]:
%%time
df = pd.read_csv(f'./{dataset}')
display(df.head())

,tweet_id,author_id,created_at,in_reply_to,text,escalate
0,2,115712,Tue Oct 31 22:11:45 +0000 2017,sprintcare,@sprintcare and how do you propose we do that,False
1,3,115712,Tue Oct 31 22:08:27 +0000 2017,sprintcare,@sprintcare I have sent several private messag...,True
2,5,115712,Tue Oct 31 21:49:35 +0000 2017,sprintcare,@sprintcare I did.,False
3,16,115713,Tue Oct 31 20:00:43 +0000 2017,sprintcare,@sprintcare Since I signed up with you....Sinc...,False
4,22,115716,Tue Oct 31 22:16:48 +0000 2017,Ask_Spectrum,@Ask_Spectrum Would you like me to email you a...,False


CPU times: user 1.67 s, sys: 162 ms, total: 1.83 s
Wall time: 1.86 s


In [9]:
print(f'Number of rows in dataset: {df.shape[0]}')
print(df['escalate'].value_counts())

Number of rows in dataset: 520793
False    417800
True     102993
Name: escalate, dtype: int64


## Part 2: Get the data into the right shape

In [10]:
train_df, val_df, _, _ = train_test_split(df, df['escalate'], test_size=0.2, random_state=0)
print(f'{train_df.shape[0]} rows in training data')
print(f'{val_df.shape[0]} rows in validation data')

416634 rows in training data
104159 rows in validation data


In [11]:
%%time

def preprocess(df):
    all_rows = df.values.tolist()
    transformed_rows = list(map(transform_instance, all_rows))
    transformed_df = pd.DataFrame(transformed_rows)
    return transformed_df

def transform_instance(row):
    text = slugify(row[4], separator=' ')
    cur_row = []
    label = "__label__1" if row[5] == True else "__label__0" # Prefix 0 or 1 from sentiment
    cur_row.append(label)
    cur_row.append(text)
    return ' '.join(cur_row)

transformed_validation_rows = preprocess(val_df)
display(transformed_validation_rows.head())

,0
0,__label__1 115990 no joke this is one of the w...
1,__label__0 amazonhelp primeira camada https t ...
2,__label__1 microsofthelps my mistake
3,__label__1 770932 americanair they notoriously...
4,__label__1 amazonhelp neither man seems to kno...


CPU times: user 6.68 s, sys: 0 ns, total: 6.68 s
Wall time: 6.78 s


In [12]:
s3_validation_data = f's3://{data_bucket}/{subfolder}/processed/validation.csv'

transformed_validation_rows.to_csv(
        f's3://{data_bucket}/{subfolder}/processed/validation.csv',header=False, index=False, quoting=csv.QUOTE_NONE, sep='|', escapechar='^')



In [13]:
%%time
transformed_train_rows = preprocess(train_df)
display(transformed_train_rows.head())

s3_train_data = f's3://{data_bucket}/{subfolder}/processed/train.csv'

transformed_train_rows.to_csv(
        f's3://{data_bucket}/{subfolder}/processed/train.csv',header=False, index=False, quoting=csv.QUOTE_NONE, sep='|', escapechar='^')


,0
0,__label__0 amazonhelp et en plus se faire engu...
1,__label__1 morrisons 641226 standard reply mor...
2,__label__1 idea cares 1936 116590 this is gr8 ...
3,__label__0 askamex yes i did weeks ago and no ...
4,__label__0 amazonhelp i don t want your stupid...


CPU times: user 28.9 s, sys: 143 ms, total: 29.1 s
Wall time: 30.5 s


## Part 3: Create training and validation datasets

In [15]:
%%time

train_data = sagemaker.inputs.TrainingInput(s3_train_data, distribution='FullyReplicated', 
                        content_type='text/plain', s3_data_type='S3Prefix')
validation_data = sagemaker.inputs.TrainingInput(s3_validation_data, distribution='FullyReplicated', 
                             content_type='text/plain', s3_data_type='S3Prefix')

CPU times: user 24 µs, sys: 0 ns, total: 24 µs
Wall time: 28.1 µs


## Part 4: Train the model

In [16]:
s3_output_location = f's3://{data_bucket}/{subfolder}/output'

sess = sagemaker.Session()

container = sagemaker.image_uris.retrieve(
            region=boto3.Session().region_name, 
            framework= "blazingtext", 
            version="1")

estimator = sagemaker.estimator.Estimator(container,
                                         role, 
                                         instance_count=1, 
                                         instance_type='ml.c5.4xlarge',
                                         max_run = 600,
                                         output_path=s3_output_location,
                                         sagemaker_session=sess)

estimator.set_hyperparameters(mode="supervised",
                            epochs=10,
                            vector_dim=10,
                            early_stopping=True,
                            patience=4,
                            min_epochs=5,
                            word_ngrams=2)

estimator.fit({'train': train_data, 'validation': validation_data})

2022-02-12 01:19:38 Starting - Starting the training job...
2022-02-12 01:19:42 Starting - Launching requested ML instancesProfilerReport-1644628778: InProgress
.........
2022-02-12 01:21:34 Starting - Preparing the instances for training......
2022-02-12 01:22:35 Downloading - Downloading input data
2022-02-12 01:22:35 Training - Downloading the training image..Arguments: train
[02/12/2022 01:22:49 WARNING 140407754389120] Loggers have already been setup.
[02/12/2022 01:22:49 WARNING 140407754389120] Loggers have already been setup.
[02/12/2022 01:22:49 INFO 140407754389120] nvidia-smi took: 0.025170087814331055 secs to identify 0 gpus
[02/12/2022 01:22:49 INFO 140407754389120] Running single machine CPU BlazingText training using supervised mode.
Number of CPU sockets found in instance is  1
[02/12/2022 01:22:49 INFO 140407754389120] Processing /opt/ml/input/data/train/train.csv . File size: 40.75419044494629 MB
[02/12/2022 01:22:49 INFO 140407754389120] Processing /opt/ml/input/data

## Part 5: Host the Model

In [17]:
endpoint_name = 'customer-support-slugify'
try:
    predictor = sagemaker.predictor.Predictor(endpoint_name=endpoint_name)
    predictor.endpoint_context()
    sess.delete_endpoint(endpoint_name)
    print('Warning: Existing endpoint deleted to make way for your new endpoint.')
except:
    pass

In [23]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

text_classifier = estimator.deploy(initial_instance_count = 1,
                                instance_type = 'ml.m5.large',
                                endpoint_name=endpoint_name,
                                serializer=JSONSerializer(),
                                deserializer=JSONDeserializer()
                                )

-----!

## Test the Model

In [36]:
tweet = "I am very happy to be finished the chapter"
tweet = "I was frustrated by my competing priorities"

tokenized_tweet = [slugify(tweet, separator=' ')]
payload = {"instances" : tokenized_tweet}
response = text_classifier.predict(payload)
escalate = pd.read_json(json.dumps(response))
escalate

,label,prob
0,[__label__1],[0.9942013621330261]


## Remove the Endpoint (optional)

Comment out this cell to remove the endpoint if you want the endpoint to exist after "run all"

In [20]:
# Remove the Endpoint (optional)
# Comment out this cell to remove the endpoint if you want the endpoint to exist after "run all"

sess.delete_endpoint(endpoint_name)